In [1]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader  # Changed from TextLoader to PyPDFLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sklearn.metrics.pairwise import cosine_similarity
from trulens_eval import TruChain, Feedback, Tru
import numpy as np
from IPython.display import display

In [2]:
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [3]:
pdf_path = "D:/TruLens/attention.pdf"  # Change this to the path of your PDF file
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"Loaded {len(documents)} pages from the PDF")

Loaded 11 pages from the PDF


In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} text chunks")

Split into 11 text chunks


In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")  # 768 dimensions
print("Embeddings model loaded")

Embeddings model loaded


In [6]:
vectorstore = Chroma.from_documents(texts, embeddings)
print("Vector store created successfully")

Vector store created successfully


In [7]:
retriever = vectorstore.as_retriever()

In [8]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("Language model loaded")

Language model loaded


In [9]:
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    device="cpu"  # Use "cuda" if you have a GPU
)

In [10]:
llm = HuggingFacePipeline(pipeline=pipe)
print("Pipeline setup complete")

Pipeline setup complete


In [11]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)
print("QA chain created")

QA chain created


In [12]:
from trulens_eval.feedback.provider.litellm import LiteLLM
provider = LiteLLM(model_engine="ollama/llama2")

In [13]:
from trulens_eval.app import App
context = App.select_context(qa_chain)

In [14]:
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
    .on(context.collect())
    .on_output()
)

f_answer_relevance = (
    Feedback(provider.relevance, name="Q/A relevance")
    .on_input_output()
)

f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name="Q/Context relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retriever.invoke.rets[:].page_content.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Q/A relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Q/A relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Q/Context relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Q/Context relevance, input context will be set to __record__.app.retriever.invoke.rets[:].page_content .


In [15]:
app_id = "RAG_QA_Chain_PDF"
tru_recorder = TruChain(
    qa_chain,
    app_id=app_id,
    feedbacks=[f_context_relevance, f_answer_relevance, f_groundedness]
)
print("TruChain created")

TruChain created


In [16]:
questions = [
    "What is the main topic of the document?",
    "Who is the author of this document?",
    # Add more questions as needed
]

In [17]:
for question in questions:
    with tru_recorder as recording:
        response = qa_chain({"query": question})
    
    print(f"\nQuestion: {question}")
    print(f"Answer: {response['result']}")
    
    # Retrieve the record of the app invocation
    rec = recording.get()  
    
    # Display the record
    display(rec)
    
    # Wait for feedback results and print them
    print("\nFeedback Results:")
    for feedback, feedback_result in rec.wait_for_feedback_results().items():
        print(f"{feedback.name}: {feedback_result.result}")



Question: What is the main topic of the document?
Answer: Attention-based neural machine translation


Record(record_id='record_hash_10f3a380a2cc7581f2e1813d4caaa91b', app_id='RAG_QA_Chain_PDF', cost=Cost(n_requests=0, n_successful_requests=0, n_classes=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, cost=0.0), perf=Perf(start_time=datetime.datetime(2024, 7, 6, 0, 24, 30, 442769), end_time=datetime.datetime(2024, 7, 6, 0, 24, 57, 460004)), ts=datetime.datetime(2024, 7, 6, 0, 24, 57, 462020), tags='-', meta=None, main_input='What is the main topic of the document?', main_output='Attention-based neural machine translation', main_error=None, calls=[RecordAppCall(call_id='91d0a916-6ee1-432b-90d8-d34a49bfd19b', stack=[RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=langchain.chains.retrieval_qa.base.RetrievalQA, id=2836511782864, init_bindings=None), name='__call__')), RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=langchain.chains.retrieval_qa.base.RetrievalQA, id=2836511782864, init_bindings=None), name='invoke')), RecordAppCallMet


Feedback Results:
Q/Context relevance: 0.9
Q/A relevance: 0.8
Groundedness: 0.9

Question: Who is the author of this document?
Answer: Nal Kalchbrenner and Stephan Gouws


Record(record_id='record_hash_50f4a5ce85a1092a30a6f84f893569bb', app_id='RAG_QA_Chain_PDF', cost=Cost(n_requests=0, n_successful_requests=0, n_classes=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, cost=0.0), perf=Perf(start_time=datetime.datetime(2024, 7, 6, 0, 33, 46, 371196), end_time=datetime.datetime(2024, 7, 6, 0, 34, 11, 161489)), ts=datetime.datetime(2024, 7, 6, 0, 34, 11, 162487), tags='-', meta=None, main_input='Who is the author of this document?', main_output='Nal Kalchbrenner and Stephan Gouws', main_error=None, calls=[RecordAppCall(call_id='2fa4a4b5-c7fe-4ca1-bf81-63511e4f7f02', stack=[RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=langchain.chains.retrieval_qa.base.RetrievalQA, id=2836511782864, init_bindings=None), name='__call__')), RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=langchain.chains.retrieval_qa.base.RetrievalQA, id=2836511782864, init_bindings=None), name='invoke')), RecordAppCallMethod(path=Len


Feedback Results:
Q/Context relevance: 0.9
Q/A relevance: 0.8
Groundedness: 0.7


In [18]:
records, feedback = tru.get_records_and_feedback(app_ids=[app_id])

In [19]:
records_np = records.to_numpy()
records_np

array([['RAG_QA_Chain_PDF',
        '{"tru_class_info": {"name": "TruChain", "module": {"package_name": "trulens_eval", "module_name": "trulens_eval.tru_chain"}, "bases": [{"name": "TruChain", "module": {"package_name": "trulens_eval", "module_name": "trulens_eval.tru_chain"}, "bases": null}, {"name": "App", "module": {"package_name": "trulens_eval", "module_name": "trulens_eval.app"}, "bases": null}, {"name": "AppDefinition", "module": {"package_name": "trulens_eval.schema", "module_name": "trulens_eval.schema.app"}, "bases": null}, {"name": "WithClassInfo", "module": {"package_name": "trulens_eval.utils", "module_name": "trulens_eval.utils.pyschema"}, "bases": null}, {"name": "SerialModel", "module": {"package_name": "trulens_eval.utils", "module_name": "trulens_eval.utils.serial"}, "bases": null}, {"name": "BaseModel", "module": {"package_name": "pydantic", "module_name": "pydantic.main"}, "bases": null}, {"name": "WithInstrumentCallbacks", "module": {"package_name": "trulens_eval",

In [20]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.3:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>